In [2]:
import json
from web3 import Web3
import os
import platform
from getpass import getpass

In [3]:
operating_system = platform.system()
d = '/'
if operating_system == 'Windows':
    d = '\\'

ganache_url = "HTTP://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# This is our main account that controls everything
web3.eth.defaultAccount = web3.eth.accounts[0]

In [4]:
web3.eth.accounts

['0xb6ef75b96D856a6cE0C796be4a7ABfB81E5D600A',
 '0xa2D0B6dbCd04a789da44CCECA9d66B6dfc93A124',
 '0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290',
 '0x9cbFF84198C1c3ed898C52Fbc0076Bf008476C4B',
 '0x18a31E094820e4C9C79a073A0927FA72635b18C7',
 '0xDED00A7f19D445B2c6D3ddFffbf78BB490771937',
 '0x37594b8C8aC1D94457C86e30dE793099CE659817',
 '0x6a1ab8E6bb7e9CC3463997FC7F38e3B3855282b5',
 '0xAfd3bB14ff7735B0D9792e5bcc9BB5e758a32ccB',
 '0x8b23fa43919cB11b9CDa04FA45B92aaA0A0Adcd8']

In [52]:
def getContract(filename):
    with open('..'+ d +'build'+ d +'contracts'+ d +filename) as contractFile:
        contractJson=json.load(contractFile)
        contractABI =contractJson['abi']
        contractAdd=web3.toChecksumAddress(contractJson['networks']['5777']['address']) 
        return contractABI, contractAdd



# Get the organization contract address
organizationContractABI, organizationContractAdd = getContract('Organization.json')	
organizationContract = web3.eth.contract(address=organizationContractAdd, abi=organizationContractABI)

#get the user contract address
userContractABI,userContractAdd = getContract('User.json')
userContract = web3.eth.contract(address=userContractAdd, abi=userContractABI)

# Get the accounts contract address
accountsContractABI,accountsContractAdd = getContract('Accounts.json')
accountsConract = web3.eth.contract(address=accountsContractAdd, abi=accountsContractABI)

loansContractABI, loansContractAddress= getContract('Loans.json')


In [ ]:
accountsConract.functions.add(web3.eth.accounts[1], False).transact()
accountsConract.functions.add(web3.eth.accounts[2], False).transact()
accountsConract.functions.add(web3.eth.accounts[5], True).transact()
accountsConract.functions.add(web3.eth.accounts[6], True).transact()

In [54]:
def deleteUser():
    accountsConract.functions.deleteAccount(web3.eth.accounts[1]).transact()

def createLoan(_address,_loaner,_amount,_loansContractAddress):
    organizationContract.functions.setLoansContractAddress(_loansContractAddress).transact();
    check=organizationContract.functions.createLoan(_address,_loaner,_amount).transact()	
    return check

In [49]:
deleteUser()

In [69]:
def getPendingLoans(_userContract, _pk=None):
    pendingLoans = _userContract.functions.getPendingLoans().buildTransaction({'gas': 70000,'gasPrice': web3.toWei('1', 'gwei'),'from': loanieAddress,'nonce': web3.eth.getTransactionCount(loanieAddress)}) 
    print(type(pendingLoans))
    private_key = getpass('Enter your key: ') if _pk is not None else _pk
    test_transaction = {'gas': 70000,'gasPrice': web3.toWei('1', 'gwei'),'from': loanieAddress,
                        'nonce': web3.eth.getTransactionCount(loanieAddress), 'data':pendingLoans['data']}
    print(type(test_transaction))
    signed_txn = web3.eth.account.signTransaction(test_transaction, private_key=private_key)
    print(signed_txn)
    transaction_hash = web3.eth.sendRawTransaction(signed_txn.rawTransaction)
    print('Transaction Block: \n', web3.eth.getTransaction(transaction_hash))
    print('Transaction Receipt: \n', web3.eth.getTransactionReceipt(transaction_hash))
    print('Pending loan ended!')

In [58]:
def getPendingLoans2(_userContract):
    tx_hash = _userContract.functions.getPendingLoans().transact()
    receipt = web3.eth.getTransactionReceipt(tx_hash)
    rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
    return rich_logs

In [59]:
# Suppose bank wants to create a loan
loanieAddress = input("Enter the addrses: ")

### Check to see whether it is a user or organization address
index = accountsConract.functions.getIndex(loanieAddress).call()

Enter the addrses: 0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290


In [60]:
index

1

In [61]:
# Second index account private key
pk = '8700bbbe5cc527282fc13aa85dfc9cbe2493cdaa4b87fea14c0b30ad56c129d7'

In [70]:
if index != -1:
    loanieType = accountsConract.functions.getType(index).call()
    if not loanieType:
        #res = organizationContract.functions.createLoan(loanieAddress).call()
        res=True
        print(res)
        choice=input("pending loans Y or N")
        if choice == "Y":
            getPendingLoans(userContract, pk)
            #getPendingLoans2(userContract)
    else :
        choice=input("create loan Y/N?")
        if choice=="Y":
            if createLoan(web3.eth.accounts[2],web3.eth.accounts[6],1000,loansContractAddress):
                print("Loan created")
else:
    print("This account is not registered in our system.")

True
pending loans Y or NY
<class 'dict'>
Enter your key: ········
<class 'dict'>
SignedTransaction(rawTransaction=HexBytes('0xf85413843b9aca0083011170808084ee24af5a1ca04597d2051fc46537e25d6ed3ffc591d8e884a9788091cce6d6f30c2ae61da986a0550a33f61b749d17b0415b0fa852207306f4bf7617970b00e3897810e2adb936'), hash=HexBytes('0x77cf9b40f8a6e732928bf33469537d3e5bdd0e8e590e7e3b2486d140d3936e77'), r=31477829963904949602242535707613059471504025820475283485160735556790862588294, s=38464619224327397238020350119479942591358129284296368379486590030116620712246, v=28)


ValueError: {'message': 'VM Exception while processing transaction: invalid opcode', 'code': -32000, 'data': {'0x77cf9b40f8a6e732928bf33469537d3e5bdd0e8e590e7e3b2486d140d3936e77': {'error': 'invalid opcode', 'program_counter': 0, 'return': '0x'}, 'stack': 'RuntimeError: VM Exception while processing transaction: invalid opcode\n    at Function.RuntimeError.fromResults (/opt/ganache/resources/static/node/node_modules/ganache-core/lib/utils/runtimeerror.js:89:13)\n    at BlockchainDouble.processBlock (/opt/ganache/resources/static/node/node_modules/ganache-core/lib/blockchain_double.js:632:24)\n    at processTicksAndRejections (internal/process/task_queues.js:93:5)', 'name': 'RuntimeError'}}